# Neurostock - Team Galaxy - Samsung Innovation Campus 2024 - 2025

## Paso 1: Preparación de los Datos

In [171]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import DataLoader, TensorDataset
data_folder = r"C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Data"
companies = [
    "AAPL", #0
    "ADBE", #1
    "AMZN", #2
    "CSCO", #3
    "DELL", #4
    "GOOGL", #5
    "IBM", #6
    "INTC", #7
    "META", #8
    "MSFT", #9
    "NOK", #10
    "NTDOY", #11
    "NVDA", #12
    "NFLX", #13
    "ORCL", #14
    "QCOM", #15
    "SONY", #16
    "SSNLF", #17
    "TSLA", #18
]

data_paths = {}

# Generación de rutas automatizada para cada Data de cada empresa
for ticker in companies:
    data_paths[ticker] = {
        "historical_data": f"{data_folder}/Historical_Data/{ticker}_historical_data.csv".replace("\\", "/"),
        "balance_sheet": f"{data_folder}/Balance_Sheet_Data/{ticker}_balance_sheet.csv".replace("\\", "/"),
        "cash_flow": f"{data_folder}/Cash_Flow_Data/{ticker}_cash_flow.csv".replace("\\", "/"),
        "income_statement": f"{data_folder}/Income_Statement_Data/{ticker}_income_statement.csv".replace("\\", "/"),
        "financial_ratios": f"{data_folder}/Financial_Ratios_Data/{ticker}_financial_ratios.csv".replace("\\", "/"),
    }

### Funciones para manejar declaraciones de Dataframes

In [172]:
def load_Historical_Data(ticker, data = data_paths):
    historical_csv_path = data_paths[ticker]["historical_data"]
    return pd.read_csv(historical_csv_path)

def load_Balance_Sheet(ticker, data = data_paths):
    Balance_Sheet_csv_path = data_paths[ticker]["balance_sheet"]
    return pd.read_csv(Balance_Sheet_csv_path)

def load_Cash_Flow(ticker, data = data_paths):
    Cash_Flow_csv_path = data_paths[ticker]["cash_flow"]
    return pd.read_csv(Cash_Flow_csv_path)

def load_Income_Statement(ticker, data = data_paths):
    Income_Statement_csv_path = data_paths[ticker]["income_statement"]
    return pd.read_csv(Income_Statement_csv_path)

def load_Financial_Ratios(ticker, data = data_paths):
    Financial_Ratios_csv_path = data_paths[ticker]["financial_ratios"]
    return pd.read_csv(Financial_Ratios_csv_path)

In [173]:
# Dataset de prueba
ticker = companies[0]
historical_data = load_Historical_Data(ticker)

### Limpieza de Dataset

In [174]:
# Verificar que las columnas sean las esperadas
expected_columns = ["Date","Open","High","Low","Close", "Volume","Dividends","Stock Splits","Adj Close"]
actual_columns = historical_data.columns.tolist()
if actual_columns == expected_columns:
    # Eliminar las columnas innecesarias
    historical_data = historical_data.drop(columns=["Dividends", "Stock Splits"])

# Ajustando el formato de las fechas
historical_data["Date"] = pd.to_datetime(historical_data["Date"], errors="coerce", utc=True)
historical_data["Date"] = historical_data["Date"].dt.tz_localize(None)
historical_data["Date"] = historical_data["Date"].dt.strftime("%Y-%m-%d")

# Limpiar Data
historical_data = historical_data.dropna()

##### Samsung Stocks KRW to USD

In [175]:
if ticker == companies[17]:
    won_to_usd_00_17 = pd.read_csv(
        r"C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Data\Won_Conversion_Data\KRW_TO_USD_2000-2017.csv"
    )
    won_to_usd_04_22 = pd.read_csv(
        r"C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Data\Won_Conversion_Data\KRW_TO_USD_2004-2022.csv"
    )
    # Preparar Datos de conversión
    won_to_usd_04_22 = won_to_usd_04_22[
        ["Date", "KRW=X"]
    ]  # dejar solamente conversion de wones
    # Borrar filas con datos perdidos
    historical_data = historical_data[historical_data["Volume"] != 0]
    won_to_usd_00_17 = won_to_usd_00_17[won_to_usd_00_17["DEXKOUS"] != "."]
    won_to_usd_04_22 = won_to_usd_04_22[won_to_usd_04_22["KRW=X"] != "."]
    # Renombrar columnas
    won_to_usd_00_17 = won_to_usd_00_17.rename(
        columns={"DEXKOUS": "Value", "DATE": "Date"}
    )
    won_to_usd_04_22 = won_to_usd_04_22.rename(columns={"KRW=X": "Value"})
    # Ajustar formato de fechas
    won_to_usd_00_17["Date"] = pd.to_datetime(won_to_usd_00_17["Date"], errors="coerce")
    won_to_usd_00_17["Date"] = won_to_usd_00_17["Date"].dt.tz_localize(None)
    won_to_usd_00_17["Date"] = won_to_usd_00_17["Date"].dt.strftime("%Y-%m-%d")
    won_to_usd_00_17["Value"] = won_to_usd_00_17["Value"].astype(float, errors="ignore")
    won_to_usd_04_22["Date"] = pd.to_datetime(won_to_usd_04_22["Date"], errors="coerce")
    won_to_usd_04_22["Date"] = won_to_usd_04_22["Date"].dt.tz_localize(None)
    won_to_usd_04_22["Date"] = won_to_usd_04_22["Date"].dt.strftime("%Y-%m-%d")
    won_to_usd_04_22["Value"] = won_to_usd_04_22["Value"].astype(float, errors="ignore")
    # Eliminar filas vacias
    won_to_usd_00_17 = won_to_usd_00_17.dropna()
    won_to_usd_04_22 = won_to_usd_04_22.dropna()
    # Unir los datasets
    won_combined = pd.concat([won_to_usd_00_17, won_to_usd_04_22])
    combined_dollar_values = won_combined.drop_duplicates(subset="Date")
    historical_data = pd.merge(
        historical_data, combined_dollar_values, on="Date", how="left"
    )
    historical_data = historical_data.dropna()
    # Convertir a USD
    historical_data["Close"] = historical_data["Close"] / historical_data["Value"]
    historical_data["Open"] = historical_data["Open"] / historical_data["Value"]
    historical_data["High"] = historical_data["High"] / historical_data["Value"]
    historical_data["Low"] = historical_data["Low"] / historical_data["Value"]
    historical_data["Adj Close"] = (
        historical_data["Adj Close"] / historical_data["Value"]
    )
    # Eliminar columna Value
    historical_data = historical_data.drop(columns=["Value"])

### Funciones para Calculo de Indices Básicos

In [176]:
# Calculo del SMA: Simple Moving Average
historical_data["SMA_5"] = historical_data["Close"].rolling(window=5).mean()
historical_data["SMA_10"] = historical_data["Close"].rolling(window=10).mean()
historical_data["SMA_20"] = historical_data["Close"].rolling(window=20).mean()
historical_data["SMA_50"] = historical_data["Close"].rolling(window=50).mean()
historical_data["SMA_100"] = historical_data["Close"].rolling(window=100).mean()
historical_data["SMA_200"] = historical_data["Close"].rolling(window=200).mean()

# Calculo del RSI: Relative Strength Index
window = 14
delta = historical_data["Close"].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
rs = gain / loss
historical_data["RSI_14"] = 100 - (100 / (1 + rs))

# Calculo del MACD: Moving Average Convergence Divergence
short_window = 12
long_window = 26
signal_window = 9

short_ema = historical_data["Close"].ewm(span=short_window, adjust=False).mean()
long_ema = historical_data["Close"].ewm(span=long_window, adjust=False).mean()
historical_data["MACD"] = short_ema - long_ema
historical_data["Signal_Line"] = (
    historical_data["MACD"].ewm(span=signal_window, adjust=False).mean()
)
historical_data = historical_data.dropna()

historical_data

,Date,Open,High,Low,Close,Volume,Adj Close,SMA_5,SMA_10,SMA_20,SMA_50,SMA_100,SMA_200,RSI_14,MACD,Signal_Line
199,2000-10-16,0.335732,0.349838,0.321625,0.323507,820176000,0.323507,0.313162,0.325011,0.529071,0.705272,0.731816,0.797268,18.023952,-0.137471,-0.124625
200,2000-10-17,0.326328,0.330089,0.296234,0.302817,601720000,0.302817,0.310905,0.321719,0.499119,0.696902,0.728279,0.794571,18.467764,-0.134389,-0.126578
201,2000-10-18,0.292472,0.316923,0.282127,0.302817,834265600,0.302817,0.312409,0.316453,0.468332,0.688889,0.724809,0.792229,9.422418,-0.130443,-0.127351
202,2000-10-19,0.288240,0.298115,0.275545,0.284949,1506724800,0.284949,0.309212,0.311751,0.439931,0.680294,0.721071,0.789742,26.609397,-0.127291,-0.127339
203,2000-10-20,0.286830,0.306578,0.284949,0.293412,791263200,0.293412,0.301500,0.307707,0.415339,0.671849,0.717685,0.787635,32.568529,-0.122695,-0.126410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6322,2025-02-21,245.949997,248.690002,245.220001,245.550003,53197400,245.550003,245.064001,239.137000,235.958032,240.473390,235.228955,223.910876,62.766179,2.288446,0.363129
6323,2025-02-24,244.929993,248.860001,244.419998,247.100006,51326400,247.100006,245.564001,241.109000,237.186266,240.485810,235.427058,224.242096,80.231290,2.668686,0.824240
6324,2025-02-25,248.000000,250.000000,244.910004,247.039993,48013300,247.039993,246.078000,243.048000,238.057889,240.476652,235.572574,224.569582,76.585168,2.931394,1.245671
6325,2025-02-26,244.330002,244.979996,239.130005,240.360001,44433600,240.360001,245.176001,243.822000,238.175973,240.359467,235.719042,224.861976,62.116289,2.570937,1.510724


### Inclusión de Data Financiera Extra

In [ ]:
market_indexes = pd.read_csv(r"C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Data\Xtra_Data\market_indexes.csv")
market_indexes["Date"] = pd.to_datetime(market_indexes["Date"], errors="coerce", utc=True)
market_indexes["Date"] = market_indexes["Date"].dt.tz_localize(None)
market_indexes["Date"] = market_indexes["Date"].dt.strftime("%Y-%m-%d")
market_indexes = market_indexes.dropna()



asian_financial_statement = pd.read_csv(r"C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Data\Xtra_Data\asian_financial_statement.csv")
asian_financial_statement = asian_financial_statement.rename(columns={"announcement date":"Date"})
asian_financial_statement["Date"] = pd.to_datetime(asian_financial_statement["Date"], errors="coerce", utc=True)
asian_financial_statement["Date"] = asian_financial_statement["Date"].dt.tz_localize(None)
asian_financial_statement["Date"] = asian_financial_statement["Date"].dt.strftime("%Y-%m-%d")
asian_financial_statement = asian_financial_statement.dropna()

In [211]:
asian_financial_statement

,Date,ticker,asset-Cash and cash equivalents,asset-Account receivable,asset-inventory,asset-total asset,asset-total asset growth rate,liability-Account payable,liability-Advance from customers,liability-total liability,liability-liability growth rate,Debt ratio,Total Investors Equity
0,2006-04-07,sz000989,4.881815e+08,1.161389e+08,1.861622e+08,1.341121e+09,1.047215,7.798643e+07,1.486313e+07,2.375994e+08,48.167748,17.716478,1.103522e+09
1,2006-04-08,sz000620,4.398212e+06,2.686289e+06,2.341095e+07,2.420870e+08,-58.050519,3.628385e+07,2.325004e+07,6.535133e+08,36.322100,269.949801,-4.114263e+08
2,2006-04-10,sz000402,1.250996e+09,4.932786e+08,5.080752e+09,8.867467e+09,47.592097,4.424058e+08,1.117493e+09,6.520724e+09,63.911966,73.535364,2.346743e+09
3,2006-04-10,sh600561,8.089855e+07,1.786341e+07,6.230296e+06,8.327838e+08,24.019395,3.638972e+07,7.914773e+06,4.177053e+08,38.231689,50.157710,4.150785e+08
4,2006-04-11,sz000690,8.670221e+07,1.926862e+08,2.331779e+08,1.980865e+09,31.451068,1.833868e+07,3.798017e+07,1.410394e+09,33.836909,71.200892,5.704716e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196475,2022-05-12,sh603117,1.952451e+08,2.920142e+08,1.687624e+08,5.573747e+09,-9.736895,1.166751e+08,3.922820e+07,3.059359e+09,-17.032592,54.888724,2.514388e+09
196477,2022-05-12,sh600679,7.017341e+08,3.726852e+08,1.917229e+08,3.384281e+09,10.034332,1.437118e+08,1.576880e+06,8.783541e+08,-22.184619,25.953932,2.505927e+09
196478,2022-05-12,sh600679,8.976623e+08,2.783644e+08,2.959343e+08,3.484441e+09,29.656775,1.891915e+08,1.662809e+06,9.981275e+08,32.154400,28.645273,2.486313e+09
196480,2022-05-13,sh688981,8.059960e+10,4.920151e+09,8.372443e+09,2.381059e+11,17.742809,3.311792e+09,1.016750e+08,7.081517e+10,25.116823,29.741040,1.672907e+11


## Paso 2: Entrenamiento de Redes Neuronales

### Preparación de Datos Históricos de Acciones

In [178]:
# Seleccionar columnas de datos históricos
historical_features = [
    "Open",
    "High",
    "Low",
    "Close",
    "Volume",
    "SMA_5",
    "SMA_10",
    "SMA_20",
    "SMA_50",
    "SMA_100",
    "SMA_200",
    "RSI_14",
    "MACD",
    "Signal_Line",
]
data_hist = historical_data[historical_features].values

# Normalizar los datos históricos
scaler_hist = MinMaxScaler(feature_range=(0, 1))
data_hist = scaler_hist.fit_transform(data_hist)

# Crear secuencias de tiempo para datos históricos
time_steps = 60
X_hist, y_hist = [], []
for i in range(len(data_hist) - time_steps):
    X_hist.append(data_hist[i : i + time_steps])
    y_hist.append(
        data_hist[i + time_steps, 3]
    )  # Usamos 'Close' como etiqueta (índice 3)
X_hist = np.array(X_hist)
y_hist = np.array(y_hist)

# Dividir los datos en conjuntos de entrenamiento y prueba
train_size_hist = int(len(X_hist) * 0.8)
X_train_hist, X_test_hist = X_hist[:train_size_hist], X_hist[train_size_hist:]
y_train_hist, y_test_hist = y_hist[:train_size_hist], y_hist[train_size_hist:]

# Convertir a tensores de PyTorch
X_train_hist = torch.tensor(X_train_hist, dtype=torch.float32)
y_train_hist = torch.tensor(y_train_hist, dtype=torch.float32)
X_test_hist = torch.tensor(X_test_hist, dtype=torch.float32)
y_test_hist = torch.tensor(y_test_hist, dtype=torch.float32)

### Preparación de Datos Macroeconómicos

In [214]:
# Seleccionar columnas de datos macroeconómicos
macro_features = [
    "Close_^GSPC",
    "Close_^IXIC",
    "High_^GSPC",
    "High_^IXIC",
    "Low_^GSPC",
    "Low_^IXIC",
    "Open_^GSPC",
    "Open_^IXIC",
    "Volume_^GSPC",
    "Volume_^IXIC",
]
data_macro = market_indexes[macro_features].values

# Normalizar los datos macroeconómicos
scaler_macro = MinMaxScaler(feature_range=(0, 1))
data_macro = scaler_macro.fit_transform(data_macro)

X_macro, y_macro_gspc, y_macro_ixic = [], [], []
for i in range(len(data_macro) - time_steps):
    X_macro.append(data_macro[i : i + time_steps])
    y_macro_gspc.append(
        data_macro[i + time_steps, 0]
    )  # 'Close_^GSPC' como etiqueta (índice 0)
    y_macro_ixic.append(
        data_macro[i + time_steps, 1]
    )  # 'Close_^IXIC' como etiqueta (índice 1)
X_macro = np.array(X_macro)
y_macro_gspc = np.array(y_macro_gspc)
y_macro_ixic = np.array(y_macro_ixic)

# Dividir los datos en conjuntos de entrenamiento y prueba
train_size_macro = int(len(X_macro) * 0.8)
X_train_macro, X_test_macro = X_macro[:train_size_macro], X_macro[train_size_macro:]
y_train_macro_gspc, y_test_macro_gspc = (
    y_macro_gspc[:train_size_macro],
    y_macro_gspc[train_size_macro:],
)
y_train_macro_ixic, y_test_macro_ixic = (
    y_macro_ixic[:train_size_macro],
    y_macro_ixic[train_size_macro:],
)

# Convertir a tensores de PyTorch
X_train_macro = torch.tensor(X_train_macro, dtype=torch.float32)
y_train_macro_ixic = torch.tensor(y_train_macro_ixic, dtype=torch.float32)
y_train_macro_gspc = torch.tensor(y_train_macro_gspc, dtype=torch.float32)
X_test_macro = torch.tensor(X_test_macro, dtype=torch.float32)
y_test_macro_ixic = torch.tensor(y_test_macro_ixic, dtype=torch.float32)
y_test_macro_gspc = torch.tensor(y_test_macro_gspc, dtype=torch.float32)

### Preparación de Datos Financieros Asiáticos

In [180]:
# Seleccionar columnas de datos financieros asiáticos
financial_features = [
    "asset-Cash and cash equivalents",
    "asset-Account receivable",
    "asset-inventory",
    "asset-total asset",
    "asset-total asset growth rate",
    "liability-Account payable",
    "liability-Advance from customers",
    "liability-total liability",
    "liability-liability growth rate",
    "Debt ratio",
    "Total Investors Equity",
]
data_financial = asian_financial_statement[financial_features].values

# Normalizar los datos financieros asiáticos
scaler_financial = MinMaxScaler(feature_range=(0, 1))
data_financial = scaler_financial.fit_transform(data_financial)

# Crear secuencias de tiempo para datos financieros asiáticos
X_financial, y_financial = [], []
for i in range(len(data_financial) - time_steps):
    X_financial.append(data_financial[i : i + time_steps])
    y_financial.append(
        data_financial[i + time_steps, 3]
    )  # Usamos 'asset-total asset' como etiqueta (índice 3)
X_financial = np.array(X_financial)
y_financial = np.array(y_financial)

# Dividir los datos en conjuntos de entrenamiento y prueba
train_size_financial = int(len(X_financial) * 0.8)
X_train_financial, X_test_financial = (
    X_financial[:train_size_financial],
    X_financial[train_size_financial:],
)
y_train_financial, y_test_financial = (
    y_financial[:train_size_financial],
    y_financial[train_size_financial:],
)

# Convertir a tensores de PyTorch
X_train_financial = torch.tensor(X_train_financial, dtype=torch.float32)
y_train_financial = torch.tensor(y_train_financial, dtype=torch.float32)
X_test_financial = torch.tensor(X_test_financial, dtype=torch.float32)
y_test_financial = torch.tensor(y_test_financial, dtype=torch.float32)

### Definición de Modelos LSTM

In [181]:
import torch.nn as nn
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(self.dropout(out[:, -1, :]))
        return out

### Entrenamiento de Histórico de Acciones

In [ ]:
# Parámetros del modelo
input_size_hist = len(historical_features)
hidden_size = 50
num_layers = 2
output_size = 1
num_epochs = 50
batch_size = 64
learning_rate = 0.001

# Definir y entrenar el modelo LSTM para datos históricos
model_hist = LSTMModel(input_size_hist, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_hist.parameters(), lr=learning_rate)

train_loader_hist = DataLoader(
    TensorDataset(X_train_hist, y_train_hist), batch_size=batch_size, shuffle=True
)
test_loader_hist = DataLoader(
    TensorDataset(X_test_hist, y_test_hist), batch_size=batch_size, shuffle=False
)

# Entrenamiento del Modelo
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            y_batch = y_batch.view(-1, 1)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

train_model(model_hist, train_loader_hist, criterion, optimizer, num_epochs)

Epoch [1/50], Loss: 0.0002
Epoch [2/50], Loss: 0.0001
Epoch [3/50], Loss: 0.0001
Epoch [4/50], Loss: 0.0001
Epoch [5/50], Loss: 0.0000
Epoch [6/50], Loss: 0.0000
Epoch [7/50], Loss: 0.0000
Epoch [8/50], Loss: 0.0000
Epoch [9/50], Loss: 0.0000
Epoch [10/50], Loss: 0.0000
Epoch [11/50], Loss: 0.0001
Epoch [12/50], Loss: 0.0001
Epoch [13/50], Loss: 0.0000
Epoch [14/50], Loss: 0.0000
Epoch [15/50], Loss: 0.0000
Epoch [16/50], Loss: 0.0000
Epoch [17/50], Loss: 0.0000
Epoch [18/50], Loss: 0.0000
Epoch [19/50], Loss: 0.0001
Epoch [20/50], Loss: 0.0000
Epoch [21/50], Loss: 0.0000
Epoch [22/50], Loss: 0.0000
Epoch [23/50], Loss: 0.0000
Epoch [24/50], Loss: 0.0000
Epoch [25/50], Loss: 0.0000
Epoch [26/50], Loss: 0.0000
Epoch [27/50], Loss: 0.0000
Epoch [28/50], Loss: 0.0000
Epoch [29/50], Loss: 0.0000
Epoch [30/50], Loss: 0.0000
Epoch [31/50], Loss: 0.0000
Epoch [32/50], Loss: 0.0000
Epoch [33/50], Loss: 0.0001
Epoch [34/50], Loss: 0.0000
Epoch [35/50], Loss: 0.0000
Epoch [36/50], Loss: 0.0001
E

### Evaluación de Modelo Histórico

In [183]:
model_hist.eval()
with torch.no_grad():
    preds = model_hist(X_test_hist).detach().cpu().numpy().flatten()
    targets = y_test_hist.view(-1, 1).detach().cpu().numpy().flatten()
    print("Ejemplo de Predicciones vs. Targets:")
    for p, t in zip(preds[:10], targets[:10]):
        print(f"Predicción: {p:.4f}, Target: {t:.4f}")

Ejemplo de Predicciones vs. Targets:
Predicción: 0.2647, Target: 0.2750
Predicción: 0.2677, Target: 0.2706
Predicción: 0.2683, Target: 0.2744
Predicción: 0.2683, Target: 0.2785
Predicción: 0.2714, Target: 0.2814
Predicción: 0.2748, Target: 0.2843
Predicción: 0.2788, Target: 0.2911
Predicción: 0.2840, Target: 0.2957
Predicción: 0.2901, Target: 0.2923
Predicción: 0.2923, Target: 0.2888


## Desnormalizar Valores

### Función para desnormalizar

In [ ]:
def denormalize_value(norm_val, scaler, feature_index, n_features):
    # norm_val: valor normalizado (o un array de valores, uno por muestra)
    # scaler: objeto MinMaxScaler que ya se ajustó con los datos
    # feature_index: el índice de la columna "Close" (en tu caso, 3)
    # n_features: número total de features (len(historical_features))
    # Si norm_val es un array 1D, lo convertimos a 2D
    norm_val = np.array(norm_val).reshape(-1, 1)
    # Creamos un array dummy de ceros con la misma cantidad de filas y n_features columnas
    dummy = np.zeros((norm_val.shape[0], n_features))
    # Asignamos los valores normalizados en la columna 'feature_index'
    dummy[:, feature_index] = norm_val[:, 0]
    # Desnormalizamos usando el inverso del scaler
    inv = scaler.inverse_transform(dummy)
    # Extraemos i.e. "Close"
    return inv[:, feature_index]

In [ ]:
# X_test_hist tiene forma: [n_samples, time_steps, num_features]
# Obtenemos el valor "Close" del último paso en cada secuencia (índice 3)
baseline_preds_norm = X_test_hist[:, -1, 3].detach().cpu().numpy()  # forma (n_samples,)
model_hist.eval()
with torch.no_grad():
    model_preds_norm = (
        model_hist(X_test_hist).detach().cpu().numpy().flatten()
    )
    y_test_norm = (
        y_test_hist.detach().cpu().numpy().flatten()
    )  # los targets en escala [0, 1]
n_features_hist = len(historical_features)

# Desnormalizamos
model_preds = denormalize_value(
    model_preds_norm, scaler_hist, feature_index=3, n_features=n_features_hist
)
y_test_actual = denormalize_value(
    y_test_norm, scaler_hist, feature_index=3, n_features=n_features_hist
)
baseline_preds = denormalize_value(
    baseline_preds_norm, scaler_hist, feature_index=3, n_features=n_features_hist
)

mse_model = np.mean((model_preds - y_test_actual) ** 2)
mse_baseline = np.mean((baseline_preds - y_test_actual) ** 2)
print(f"MSE Modelo LSTM (desnormalizado): {mse_model:.4f}")
print(f"MSE Línea Base (desnormalizado): {mse_baseline:.4f}")

MSE Modelo LSTM (desnormalizado): 531.6965
MSE Línea Base (desnormalizado): 7.4420


### Graficar y Visualizar

In [192]:
test_dates = historical_data["Date"].iloc[train_size_hist + time_steps :].values
import plotly.graph_objects as go

test_dates_str = [str(date) for date in test_dates]

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=test_dates_str, y=y_test_actual, mode="lines+markers", name="Real")
)

fig.add_trace(
    go.Scatter(
        x=test_dates_str, y=model_preds, mode="lines+markers", name="Predicción LSTM"
    )
)

fig.add_trace(
    go.Scatter(
        x=test_dates_str, y=baseline_preds, mode="lines+markers", name="Línea Base"
    )
)

fig.update_layout(
    title="Comparación: Modelo LSTM vs. Línea Base vs. Valores Reales",
    xaxis_title="Fecha",
    yaxis_title="Precio de cierre (desnormalizado)",
    template="plotly_white",
)

fig.show()

# Guardamos el Modelo antes de subir a repositorio

In [ ]:
import os

directory = r"C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Models"
model_filename = "modelo_lstm_historico.pth"
optimizer_filename = "optimizer_lstm_historico.pth"

model_save_path = os.path.join(directory, model_filename)
optimizer_save_path = os.path.join(directory, optimizer_filename)

torch.save(model_hist.state_dict(), model_save_path)
torch.save(optimizer.state_dict(), optimizer_save_path)

print(f"Modelo guardado en {model_save_path}")
print(f"Estado del optimizador guardado en {optimizer_save_path}")

Modelo guardado en C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Models\modelo_lstm_historico.pth
Estado del optimizador guardado en C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Models\optimizer_lstm_historico.pth


### Entrenamiento de Datos Macroeconómicos

In [215]:
# Parámetros del modelo
input_size_macro = len(macro_features)
hidden_size = 50
num_layers = 2
output_size = 2  # Para predecir dos salidas, Close_^GSPC y Close_^IXIC
num_epochs = 50
batch_size = 32
learning_rate = 0.001

# Definir el modelo LSTM para datos macroeconómicos
model_macro = LSTMModel(input_size_macro, hidden_size, num_layers, output_size)

# Definir la función de pérdida y el optimizador
criterion = nn.MSELoss()
optimizer_macro = torch.optim.Adam(model_macro.parameters(), lr=learning_rate)

# Crear los DataLoader para entrenamiento y prueba
train_loader_macro = DataLoader(
    TensorDataset(X_train_macro, y_train_macro_gspc, y_train_macro_ixic),
    batch_size=batch_size,
    shuffle=True,
)
test_loader_macro = DataLoader(
    TensorDataset(X_test_macro, y_test_macro_gspc, y_test_macro_ixic),
    batch_size=batch_size,
    shuffle=False,
)


# --- Función de entrenamiento ---
def train_model_multioutput(model, train_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_gspc_batch, y_ixic_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            y_gspc_batch = y_gspc_batch.view(-1, 1)
            y_ixic_batch = y_ixic_batch.view(-1, 1)
            loss_gspc = criterion(outputs[:, 0].view(-1, 1), y_gspc_batch)
            loss_ixic = criterion(outputs[:, 1].view(-1, 1), y_ixic_batch)
            loss = loss_gspc + loss_ixic  # Total loss
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


# --- Entrenar el modelo de datos macroeconómicos ---
train_model_multioutput(
    model_macro, train_loader_macro, criterion, optimizer_macro, num_epochs
)

# --- Evaluar el modelo con el conjunto de prueba ---
model_macro.eval()
with torch.no_grad():
    test_predictions_macro = model_macro(X_test_macro)
    y_test_macro_gspc_reshaped = y_test_macro_gspc.view(-1, 1)
    y_test_macro_ixic_reshaped = y_test_macro_ixic.view(-1, 1)
    test_loss_gspc = criterion(
        test_predictions_macro[:, 0].view(-1, 1), y_test_macro_gspc_reshaped
    )
    test_loss_ixic = criterion(
        test_predictions_macro[:, 1].view(-1, 1), y_test_macro_ixic_reshaped
    )
    total_test_loss = test_loss_gspc + test_loss_ixic
    print(f"Macro Test MSE Total: {total_test_loss.item():.4f}")

Epoch [1/50], Loss: 0.0015
Epoch [2/50], Loss: 0.0014
Epoch [3/50], Loss: 0.0012
Epoch [4/50], Loss: 0.0007
Epoch [5/50], Loss: 0.0010
Epoch [6/50], Loss: 0.0005
Epoch [7/50], Loss: 0.0005
Epoch [8/50], Loss: 0.0009
Epoch [9/50], Loss: 0.0009
Epoch [10/50], Loss: 0.0011
Epoch [11/50], Loss: 0.0007
Epoch [12/50], Loss: 0.0004
Epoch [13/50], Loss: 0.0007
Epoch [14/50], Loss: 0.0006
Epoch [15/50], Loss: 0.0007
Epoch [16/50], Loss: 0.0011
Epoch [17/50], Loss: 0.0002
Epoch [18/50], Loss: 0.0002
Epoch [19/50], Loss: 0.0003
Epoch [20/50], Loss: 0.0005
Epoch [21/50], Loss: 0.0003
Epoch [22/50], Loss: 0.0007
Epoch [23/50], Loss: 0.0007
Epoch [24/50], Loss: 0.0007
Epoch [25/50], Loss: 0.0004
Epoch [26/50], Loss: 0.0006
Epoch [27/50], Loss: 0.0005
Epoch [28/50], Loss: 0.0004
Epoch [29/50], Loss: 0.0007
Epoch [30/50], Loss: 0.0003
Epoch [31/50], Loss: 0.0003
Epoch [32/50], Loss: 0.0007
Epoch [33/50], Loss: 0.0003
Epoch [34/50], Loss: 0.0005
Epoch [35/50], Loss: 0.0005
Epoch [36/50], Loss: 0.0004
E

### Desnormalizar Macro

In [216]:
baseline_preds_gspc_norm = (
    X_test_macro[:, -1, 0].detach().cpu().numpy()
)  # Forma: (n_samples,)
baseline_preds_ixic_norm = (
    X_test_macro[:, -1, 1].detach().cpu().numpy()
)  # Forma: (n_samples,)

# 2. Obtener las predicciones del modelo:
model_macro.eval()
with torch.no_grad():
    # Suponiendo que model_macro devuelve una salida de forma [n_samples, 2],
    # donde el índice 0 corresponde a Close_^GSPC y 1 a Close_^IXIC.
    test_predictions_macro = model_macro(X_test_macro)
    model_preds_gspc_norm = (
        test_predictions_macro[:, 0].detach().cpu().numpy().flatten()
    )
    model_preds_ixic_norm = (
        test_predictions_macro[:, 1].detach().cpu().numpy().flatten()
    )

    # Además, obtener los targets (en escala normalizada) ya preparados:
    y_test_macro_gspc_norm = y_test_macro_gspc.detach().cpu().numpy().flatten()
    y_test_macro_ixic_norm = y_test_macro_ixic.detach().cpu().numpy().flatten()

# Número total de features en los datos macro (para desnormalización)
n_features_macro = len(macro_features)

In [217]:
# 3. Desnormalizar las predicciones, los targets y la línea base para "Close_^GSPC" (índice 0)
model_preds_gspc = denormalize_value(
    model_preds_gspc_norm, scaler_macro, feature_index=0, n_features=n_features_macro
)
y_test_macro_gspc_actual = denormalize_value(
    y_test_macro_gspc_norm, scaler_macro, feature_index=0, n_features=n_features_macro
)
baseline_preds_gspc = denormalize_value(
    baseline_preds_gspc_norm, scaler_macro, feature_index=0, n_features=n_features_macro
)

# 4. Desnormalizar para "Close_^IXIC" (índice 1)
model_preds_ixic = denormalize_value(
    model_preds_ixic_norm, scaler_macro, feature_index=1, n_features=n_features_macro
)
y_test_macro_ixic_actual = denormalize_value(
    y_test_macro_ixic_norm, scaler_macro, feature_index=1, n_features=n_features_macro
)
baseline_preds_ixic = denormalize_value(
    baseline_preds_ixic_norm, scaler_macro, feature_index=1, n_features=n_features_macro
)

# 5. Calcular los MSE para cada target:
mse_model_gspc = np.mean((model_preds_gspc - y_test_macro_gspc_actual) ** 2)
mse_baseline_gspc = np.mean((baseline_preds_gspc - y_test_macro_gspc_actual) ** 2)

mse_model_ixic = np.mean((model_preds_ixic - y_test_macro_ixic_actual) ** 2)
mse_baseline_ixic = np.mean((baseline_preds_ixic - y_test_macro_ixic_actual) ** 2)

print(f"MSE Modelo LSTM GSPC (desnormalizado): {mse_model_gspc:.4f}")
print(f"MSE Línea Base GSPC (desnormalizado): {mse_baseline_gspc:.4f}")
print(f"MSE Modelo LSTM IXIC (desnormalizado): {mse_model_ixic:.4f}")
print(f"MSE Línea Base IXIC (desnormalizado): {mse_baseline_ixic:.4f}")

MSE Modelo LSTM GSPC (desnormalizado): 6011.4969
MSE Línea Base GSPC (desnormalizado): 2254.7485
MSE Modelo LSTM IXIC (desnormalizado): 112024.9674
MSE Línea Base IXIC (desnormalizado): 38990.9004


### Graficar Macro Data

In [218]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Crear una figura con dos subplots (una fila para cada índice)
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, subplot_titles=["Close_^GSPC", "Close_^IXIC"]
)

# Agregar las trazas para "Close_^GSPC" en la primera subgráfica (fila 1)
fig.add_trace(
    go.Scatter(
        x=test_dates_str,
        y=y_test_macro_gspc_actual,
        mode="lines+markers",
        name="Real GSPC",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=test_dates_str,
        y=model_preds_gspc,
        mode="lines+markers",
        name="Predicción LSTM GSPC",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=test_dates_str,
        y=baseline_preds_gspc,
        mode="lines+markers",
        name="Línea Base GSPC",
    ),
    row=1,
    col=1,
)

# Agregar las trazas para "Close_^IXIC" en la segunda subgráfica (fila 2)
fig.add_trace(
    go.Scatter(
        x=test_dates_str,
        y=y_test_macro_ixic_actual,
        mode="lines+markers",
        name="Real IXIC",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=test_dates_str,
        y=model_preds_ixic,
        mode="lines+markers",
        name="Predicción LSTM IXIC",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=test_dates_str,
        y=baseline_preds_ixic,
        mode="lines+markers",
        name="Línea Base IXIC",
    ),
    row=2,
    col=1,
)

# Actualizar el layout
fig.update_layout(
    height=800,
    title_text="Comparación: Predicciones vs. Valores Reales (Datos Macroeconómicos)",
    template="plotly_white",
)

fig.update_xaxes(title_text="Fecha", row=2, col=1)
fig.update_yaxes(title_text="Precio de cierre (desnormalizado)", row=1, col=1)
fig.update_yaxes(title_text="Precio de cierre (desnormalizado)", row=2, col=1)

fig.show()